# Lab 3: Знайомство з нейромережами
### Шевченко Юлія, ФІ-31мн

### Lab Task:

1. _**Повнозв'язані нейронні мережі**_. Вирішити завдання класифікації даних, з якими працювали в лабораторній №1 за допомогою повнозв’язаної нейромережі прямого поширення (fully connected feed-forward network). Результати порівняти з одержаними раніше.
2. _**Згорткові нейронні мережі**_. Вирішити завдання класифікації зображень за допомогою згорткової (convolutional) нейромережі (якщо в обраному датасеті класів забагато, достатньо залишити 3-5).
3. _**Рекурентні нейронні мережі**_. Вирішити задачу класифікації текстів (з якими працювали в лабораторній №2) за допомогою рекурентної нейромережі. Результати порівняти з одержаними раніше. 

In [1]:
# All the imports for the task
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping
from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import train_test_split

### Fully connected feed-forward network

In [2]:
# Loading dataset from csv
spotify_data = pd.read_csv('../Lab_3/song_data.csv')

In [3]:
# Showing the dataset
spotify_data.head()

,song_name,song_popularity,song_duration_ms,acousticness,danceability,energy,instrumentalness,key,liveness,loudness,audio_mode,speechiness,tempo,time_signature,audio_valence
0,Boulevard of Broken Dreams,73,262333,0.005520,0.496,0.682,0.000029,8,0.0589,-4.095,1,0.0294,167.060,4,0.474
1,In The End,66,216933,0.010300,0.542,0.853,0.000000,3,0.1080,-6.407,0,0.0498,105.256,4,0.370
2,Seven Nation Army,76,231733,0.008170,0.737,0.463,0.447000,0,0.2550,-7.828,1,0.0792,123.881,4,0.324
3,By The Way,74,216933,0.026400,0.451,0.970,0.003550,0,0.1020,-4.938,1,0.1070,122.444,4,0.198
4,How You Remind Me,56,223826,0.000954,0.447,0.766,0.000000,10,0.1130,-5.065,1,0.0313,172.011,4,0.574


In [4]:
# Showing the dataset columns
print('The dataset contains such columns: {}'.format(spotify_data.columns))

The dataset contains such columns: Index(['song_name', 'song_popularity', 'song_duration_ms', 'acousticness',
       'danceability', 'energy', 'instrumentalness', 'key', 'liveness',
       'loudness', 'audio_mode', 'speechiness', 'tempo', 'time_signature',
       'audio_valence'],
      dtype='object')


In [5]:
# Some columns have 'int64' type, so we convert them to float
spotify_data.song_duration_ms = spotify_data.song_duration_ms.astype(float)
spotify_data.audio_mode = spotify_data.audio_mode.astype(float)
spotify_data.time_signature = spotify_data.time_signature.astype(float)

In [6]:
# Considering song is popular if song_popularity value is >= 70
spotify_data['popularity'] = [1 if value >= 70 else 0 for value in spotify_data.song_popularity]
spotify_data.popularity.value_counts()

popularity
0    14384
1     4451
Name: count, dtype: int64

In [7]:
# Choosing numeric data and describing correlation
spotify_data_numeric = spotify_data.select_dtypes(include = ['float64', 'int64'])
data_correlation = spotify_data_numeric.corr()
data_correlation

,song_popularity,song_duration_ms,acousticness,danceability,energy,instrumentalness,key,liveness,loudness,audio_mode,speechiness,tempo,time_signature,audio_valence,popularity
song_popularity,1.000000,-0.018899,-0.065181,0.104290,0.001365,-0.130907,-0.013160,-0.038937,0.099442,-0.004969,0.021479,-0.022672,0.034983,-0.052895,0.660385
song_duration_ms,-0.018899,1.000000,-0.097882,-0.104985,0.092688,-0.015188,-0.004615,0.016086,0.016469,-0.026328,-0.083447,0.012791,0.000264,-0.063421,0.005092
acousticness,-0.065181,-0.097882,1.000000,-0.182500,-0.662639,0.173098,-0.002025,-0.082537,-0.557744,0.064100,-0.093574,-0.136421,-0.157177,-0.121670,-0.101694
danceability,0.104290,-0.104985,-0.182500,1.000000,0.044373,-0.130483,0.007723,-0.090694,0.177639,-0.106539,0.213728,-0.121286,0.136391,0.332014,0.117811
energy,0.001365,0.092688,-0.662639,0.044373,1.000000,-0.205914,0.020416,0.167143,0.755516,-0.051605,0.062053,0.162448,0.146265,0.316742,0.042833
instrumentalness,-0.130907,-0.015188,0.173098,-0.130483,-0.205914,1.000000,-0.009463,-0.032887,-0.390248,-0.014983,-0.081138,-0.036934,-0.069181,-0.178570,-0.152954
key,-0.013160,-0.004615,-0.002025,0.007723,0.020416,-0.009463,1.000000,-0.008193,0.010452,-0.169848,0.033639,0.003239,-0.008282,0.025356,-0.021929
liveness,-0.038937,0.016086,-0.082537,-0.090694,0.167143,-0.032887,-0.008193,1.000000,0.102450,-0.001035,0.090845,0.031876,0.013289,0.013789,-0.025308
loudness,0.099442,0.016469,-0.557744,0.177639,0.755516,-0.390248,0.010452,0.102450,1.000000,-0.058506,0.078164,0.129938,0.119241,0.199696,0.145254
audio_mode,-0.004969,-0.026328,0.064100,-0.106539,-0.051605,-0.014983,-0.169848,-0.001035,-0.058506,1.000000,-0.111627,0.020448,-0.024042,-0.004558,-0.019096


In [8]:
# Splitting data to train and test with 20% test data size
X = spotify_data_numeric.drop(['popularity'], axis=1)
y = spotify_data_numeric['popularity']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [9]:
# Standardize the variables
scaler = StandardScaler()
scaler.fit(X_train)
scaled_X_train = scaler.transform(X_train)
scaled_X_test = scaler.transform(X_test)

In [10]:
# Creating model
model = Sequential()
model.add(Dense(64, input_dim=scaled_X_train.shape[1], activation='relu'))  # First connected layer
model.add(Dense(32, activation='relu'))  # Second connected layer
model.add(Dense(1, activation='sigmoid'))  # Sigmoid output layer for binary classification

# Compile model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [11]:
# Train model
early_stopping = EarlyStopping(monitor='val_accuracy', patience=5, verbose=1, restore_best_weights=True)
model.fit(scaled_X_train, y_train, epochs=100, batch_size=32, validation_data=(scaled_X_test, y_test), callbacks=[early_stopping])

Epoch 1/100
471/471 [==============================] - 1s 1ms/step - loss: 0.2144 - accuracy: 0.9105 - val_loss: 0.0767 - val_accuracy: 0.9753
Epoch 2/100
471/471 [==============================] - 1s 1ms/step - loss: 0.0588 - accuracy: 0.9796 - val_loss: 0.0453 - val_accuracy: 0.9865
Epoch 3/100
471/471 [==============================] - 1s 1ms/step - loss: 0.0400 - accuracy: 0.9863 - val_loss: 0.0346 - val_accuracy: 0.9907
Epoch 4/100
471/471 [==============================] - 1s 1ms/step - loss: 0.0293 - accuracy: 0.9898 - val_loss: 0.0330 - val_accuracy: 0.9886
Epoch 5/100
471/471 [==============================] - 1s 1ms/step - loss: 0.0249 - accuracy: 0.9916 - val_loss: 0.0253 - val_accuracy: 0.9926
Epoch 6/100
471/471 [==============================] - 1s 1ms/step - loss: 0.0206 - accuracy: 0.9928 - val_loss: 0.0239 - val_accuracy: 0.9904
Epoch 7/100
471/471 [==============================] - 1s 1ms/step - loss: 0.0185 - accuracy: 0.9945 - val_loss: 0.0200 - val_accuracy: 0.9936

In [12]:
# Evaluate model
loss, accuracy = model.evaluate(scaled_X_test, y_test)
print(f'Test accuracy: {accuracy}')

118/118 [==============================] - 0s 853us/step - loss: 0.0200 - accuracy: 0.9936
Test accuracy: 0.9936288595199585
